In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

In [2]:
#MNISTのデータセットをダウンロードする
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
num_units = 1024

x = tf.placeholder(tf.float32, [None, 784])

#隠れ層の出力hidden1の計算
w1 = tf.Variable(tf.truncated_normal([784, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1)

#隠れ層の出力から、ソフトマックス関数を用いて確率を計算する部分
w0 = tf.Variable(tf.zeros([num_units, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [4]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
#トレーニングアルゴリズムtrain_stepの定義
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
#正解率accuracyの定義
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
#セッションの用意
sess = tf.InteractiveSession()
#Variableの初期化
sess.run(tf.initialize_all_variables())

In [7]:
#パラメータの最適化処理（勾配降下法によるパラメータの最適化）
i = 0
#2000回繰り返す
for _ in range(2000):
    i += 1
    #トレーニングセットから100個分のデータを取り出す。
    batch_xs, batch_ts = mnist.train.next_batch(100)
    #そのデータを用いて、+ 
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t: mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

Step: 100, Loss: 1067.438477, Accuracy: 0.969400
Step: 200, Loss: 979.750488, Accuracy: 0.970600
Step: 300, Loss: 909.228760, Accuracy: 0.972800
Step: 400, Loss: 945.900330, Accuracy: 0.971900
Step: 500, Loss: 933.217834, Accuracy: 0.973000
Step: 600, Loss: 981.230103, Accuracy: 0.970800
Step: 700, Loss: 880.398499, Accuracy: 0.974100
Step: 800, Loss: 1008.995789, Accuracy: 0.971500
Step: 900, Loss: 933.573181, Accuracy: 0.973400
Step: 1000, Loss: 912.573853, Accuracy: 0.972500
Step: 1100, Loss: 1027.446167, Accuracy: 0.969800
Step: 1200, Loss: 1015.711914, Accuracy: 0.972900
Step: 1300, Loss: 1096.841919, Accuracy: 0.970400
Step: 1400, Loss: 948.667969, Accuracy: 0.974400
Step: 1500, Loss: 972.939331, Accuracy: 0.974000
Step: 1600, Loss: 1030.249878, Accuracy: 0.972600
Step: 1700, Loss: 1045.442261, Accuracy: 0.972500
Step: 1800, Loss: 1058.476685, Accuracy: 0.972500
Step: 1900, Loss: 1049.189575, Accuracy: 0.972000
Step: 2000, Loss: 968.911255, Accuracy: 0.973800


In [ ]:
images, labels = mnist.test.images, mnist.test.labels
p_val = sess.run(p, feed_dict={x:images, t: labels}) 

fig = plt.figure(figsize=(8,15))
for i in range(10):
    c = 1
    for (image, label, pred) in zip(images, labels, p_val):
        prediction, actual = np.argmax(pred), np.argmax(label)
        if prediction != i:
            continue
        if (c < 4 and i == actual) or (c >= 4 and i != actual):
            subplot = fig.add_subplot(10,6,i*6+c)
            subplot.set_xticks([])
            subplot.set_yticks([])
            subplot.set_title('%d / %d' % (prediction, actual))
            subplot.imshow(image.reshape((28,28)), vmin=0, vmax=1,
                           cmap=plt.cm.gray_r, interpolation="nearest")
            c += 1
            if c > 6:
                break
plt.show()